# **Homework 1 - task 28**
The main goal of this homework is to transform existing evaluation datasets into a format suitable for evaluating the linguistic skills of Large Language Models (LLMs) by reframing tasks as multi-choice Question Answering (QA) tasks, providing effective prompts, and generating distractors where necessary, all formatted in JSON Lines standard for submission.

In [244]:
# from google.colab import drive
# drive.mount('/content/drive')

# **1. Data Loading:**


First we need to download the datasets from evalita to our python eviroment



In [245]:
##### Data Reader -- English/French/Italian
!git clone https://github.com/shammur/SemEval2022Task3.git
%cd ./SemEval2022Task3/
#data/train/train_subtask-2/
!ls

c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\Subtask_2\SemEval2022Task3


fatal: destination path 'SemEval2022Task3' already exists and is not an empty directory.
'ls' is not recognized as an internal or external command,
operable program or batch file.


# Sub Task 2

**Regression sub-task**, which consists in predicting the average score assigned by human annotators on a seven point Likert-scale with respect to the subset of data evaluated via crowdsourcing.

## 1.1 visualize initial data

**dev.xml** is the training (development) dataset.


**test_gold.xml** is the test dataset.

In [246]:
import os

# Change the working directory
os.chdir(r'c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\Subtask_2')

# Verify the current working directory
print("Current working directory:", os.getcwd())



Current working directory: c:\Users\35193\Desktop\Sapienza\1 year\Second Semester\Multilingual Natural Language Processing\Homework 1\GitHub\NLP_Sapienza\Task_28\Subtask_2


In [247]:
def visualize_original_data(file_path):
    try:
        with open(file_path, 'r') as file:
            for i in range(5):  
                line = file.readline()
                print(line.rstrip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Check test data
file_path = r'SemEval2022Task3\data\test\official_test_set_with_labels\subtask-2\It-Subtask2-scores.tsv'
visualize_original_data(file_path)


ID	Construction	Sentence	Scores
it_0	ingeneral	Amo gli arbusti, e le querce in generale.	2.91
it_1	particular	Amo le montagne, e in particolare i centri commerciali.	1.3
it_2	comparatives	Amo le foreste più delle case.	5.92
it_3	andtoo	Amo le foreste, ed anche i centri commerciali.	5.6


In [248]:
# join the fold training data to a single file

def join_tsv_files(file_paths, output_file):
    with open(output_file, 'w') as outfile:
        for file_path in file_paths:
            with open(file_path, 'r') as infile:
                for line in infile:
                    outfile.write(line)

# Example usage
file1 = r'SemEval2022Task3\data\train\train_subtask-2\it\It-Subtask2-fold_0.tsv'
file2 = r'SemEval2022Task3\data\train\train_subtask-2\it\It-Subtask2-fold_1.tsv'
output_file = 'It-Subtask2-train.tsv'

join_tsv_files([file1, file2], output_file)


In [249]:
# Check info in train data
file_path = r'It-Subtask2-train.tsv'
visualize_original_data(file_path)

ID	Sentence	Score
73	Adoro gli abeti, ma non i cespugli.	4.33
106	Adoro il vitello, ma non i granchi.	3.33
203	Adoro gli animali, un tipo interessante di abete.	1.25
466	Amo gli abeti più dei cespugli.	5.18


## 1.2 Data Reframing:
  First we need to change the type of the file from xml to json. Then we want to convert that json into the following format:
```JSON
{
    "id":       int,
    "text":     str,
    "choices":  list[str|int],
    "label":    int
}
```

In [250]:
import csv
import json

def tsv_to_jsonl(input_file, output_file):
    with open(input_file, 'r', newline='') as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t')
        data = list(reader)

    with open(output_file, 'w') as jsonfile:
        json.dump(data, jsonfile, indent=4)

tsv_file = r'SemEval2022Task3\data\test\official_test_set_with_labels\subtask-2\It-Subtask2-scores.tsv'
jsonl_file = 'It-Subtask2-test.jsonl'
tsv_to_jsonl(tsv_file, jsonl_file)


In [251]:
tsv_file = 'It-Subtask2-train.tsv'
jsonl_file = 'It-Subtask2-train.jsonl'
tsv_to_jsonl(tsv_file, jsonl_file)

## 1.3 Visualize jsonl information

In [252]:
import json
import json

def visualize_jsonl_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        
    for entry in data[:3]:  
        print("{")
        for key, value in entry.items():
            print(f'    "{key}": "{value}",')
        print("},")

json_file = 'It-Subtask2-test.jsonl'
visualize_jsonl_data(json_file)


{
    "ID": "it_0",
    "Construction": "ingeneral",
    "Sentence": "Amo gli arbusti, e le querce in generale.",
    "Scores": "2.91",
},
{
    "ID": "it_1",
    "Construction": "particular",
    "Sentence": "Amo le montagne, e in particolare i centri commerciali.",
    "Scores": "1.3",
},
{
    "ID": "it_2",
    "Construction": "comparatives",
    "Sentence": "Amo le foreste più delle case.",
    "Scores": "5.92",
},


In [253]:
json_file = 'It-Subtask2-train.jsonl'
visualize_jsonl_data(json_file)

{
    "ID": "73",
    "Sentence": "Adoro gli abeti, ma non i cespugli.",
    "Score": "4.33",
},
{
    "ID": "106",
    "Sentence": "Adoro il vitello, ma non i granchi.",
    "Score": "3.33",
},
{
    "ID": "203",
    "Sentence": "Adoro gli animali, un tipo interessante di abete.",
    "Score": "1.25",
},



## 1.4 Format jsonl

In order to get the json file with the desired format:

1. Rename ID to id. (int)
2. Rename Sentence to text.  (str)
3. Add a choices key with 4 of the 7  possible labels as values. One must be correct.
4. Rename Labels to label (to the position of the correct choice).
5. Delete Contruction.


In [254]:
import json
import random

def rearrange_json(file_path):
    # Mapping scores to Italian descriptors used in Likert scales
    score_to_descriptor = {
        1: "Pessimo",
        2: "Scarso",
        3: "Sufficiente",
        4: "Buono",
        5: "Molto buono",
        6: "Ottimo",
        7: "Eccellente"
    }

    with open(file_path, "r") as file:
        data = json.load(file)

        rearranged_data = []
        for pair in data:
            if 'ID' in pair and 'Sentence' in pair:
                score_key = 'Scores' if 'Scores' in pair else 'Score'
                try:
                    score = int(round(float(pair[score_key])))
                except ValueError:
                    print(f"Invalid score value")
                    continue
                new_pair = {
                    "id": int(pair['ID'].split('_')[-1]),  # Extracting the integer part of the ID
                    "text": pair['Sentence'],
                    "choices": [],  
                    "label": None  
                }
                # Generating choices based on list of descriptors
                choices = [score_to_descriptor[i] for i in range(1, 8) if i != score]
                # Ensure unique choices including the correct answer
                choices = random.sample(choices, 3)
                choices.append(score_to_descriptor[score])
                # Shuffle the choices to randomize the position of the correct answer
                random.shuffle(choices)
                new_pair["choices"] = choices
                # Generate label within the range of choices
                new_pair["label"] = new_pair["choices"].index(score_to_descriptor[score])
                rearranged_data.append(new_pair)
    # Write rearranged data back to the file
    with open(file_path, "w") as file:
        json.dump(rearranged_data, file, indent=2)

file_path_test_gold = 'It-Subtask2-test.jsonl'
rearrange_json(file_path_test_gold)


file_path_dev = 'It-Subtask2-train.jsonl'
rearrange_json(file_path_dev)

Invalid score value


## 1.5 Visualize final jsonl

In [255]:
json_file = 'It-Subtask2-test.jsonl'
visualize_jsonl_data(json_file)

{
    "id": "0",
    "text": "Amo gli arbusti, e le querce in generale.",
    "choices": "['Molto buono', 'Pessimo', 'Eccellente', 'Sufficiente']",
    "label": "3",
},
{
    "id": "1",
    "text": "Amo le montagne, e in particolare i centri commerciali.",
    "choices": "['Eccellente', 'Ottimo', 'Pessimo', 'Molto buono']",
    "label": "2",
},
{
    "id": "2",
    "text": "Amo le foreste più delle case.",
    "choices": "['Buono', 'Ottimo', 'Scarso', 'Sufficiente']",
    "label": "1",
},


In [256]:
json_file = 'It-Subtask2-train.jsonl'
visualize_jsonl_data(json_file)


{
    "id": "73",
    "text": "Adoro gli abeti, ma non i cespugli.",
    "choices": "['Scarso', 'Pessimo', 'Molto buono', 'Buono']",
    "label": "3",
},
{
    "id": "106",
    "text": "Adoro il vitello, ma non i granchi.",
    "choices": "['Scarso', 'Pessimo', 'Sufficiente', 'Eccellente']",
    "label": "2",
},
{
    "id": "203",
    "text": "Adoro gli animali, un tipo interessante di abete.",
    "choices": "['Ottimo', 'Eccellente', 'Pessimo', 'Molto buono']",
    "label": "2",
},


## **2. Promt formulation**

In order to use this dataset we need to generate three prompts that can be used to get if a text is entailed to a hypothesis and then insert them into a json file. The three prompts are:

**Template 1:**


Prompt: "Valuta se il seguente testo è accettabile o meno secondo il contesto: {{text}}"

Translation: "Evaluate whether the following text is acceptable or not according to the context: {{text}}"

**Template 2:**

Prompt: "Determina se la seguente frase è coerente con il contesto: {{text}}"

Translation: "Determine if the following sentence is coherent with the context: {{text}}"

**Template 3:**

Prompt: "Decidi se il testo fornito è congruente con la situazione descritta: {{text}}"

Translation: "Decide if the provided text is congruent with the described situation: {{text}}"


In [268]:
import json

# Function to generate a JSON Lines file with a list of prompts
def generate_json(prompts, output_file):
    data = []
    for prompt in prompts:
        data.append({"prompt": prompt})

    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    with open(output_file, 'r', encoding='utf-8') as json_file:
        print(json_file.read())


prompts = [
    "Qual è il tuo giudizio sull'accettabilità di {{text}}, su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente').",
    "Considerando {{text}}, quanto lo trovi accettabile? Esprimi la tua valutazione su una scala da 1 a 7, dove '1' corrisponde a 'Pessimo' e '7' a 'Eccellente'.",
    "Su una scala da 1 a 7, con '1' rappresentante 'Pessimo' e '7' 'Eccellente', quanto accettabile trovi {{text}}?"
]

output_file = "prompts.jsonl"
generate_json(prompts, output_file)
print(f"JSON file '{output_file}' generated successfully with {len(prompts)} prompts.")



[
    {
        "prompt": "Qual \u00e8 il tuo giudizio sull'accettabilit\u00e0 di {{text}}, su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente')."
    },
    {
        "prompt": "Considerando {{text}}, quanto lo trovi accettabile? Esprimi la tua valutazione su una scala da 1 a 7, dove '1' corrisponde a 'Pessimo' e '7' a 'Eccellente'."
    },
    {
        "prompt": "Su una scala da 1 a 7, con '1' rappresentante 'Pessimo' e '7' 'Eccellente', quanto accettabile trovi {{text}}?"
    }
]
JSON file 'prompts.jsonl' generated successfully with 3 prompts.


# **3. Llama 2 set up**

In [269]:
# Device agnostic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [270]:
# Configuration
runtimeFlag = device #Run on GPU (you can't run GPTQ on cpu)
cache_dir = None # by default, don't set a cache directory. This is automatically updated if you connect Google Drive.
scaling_factor = 1.0 # allows for a max sequence length of 16384*6 = 98304! Unfortunately, requires Colab Pro and a V100 or A100 to have sufficient RAM.

## 3.1 Install

In [271]:
!pip install -q -U transformers peft accelerate optimum

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [272]:
!pip install -q -U pdfminer.six # could maybe add pre-built wheels to speed this up.

ERROR: Invalid requirement: '#'


In [273]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

##3.2 Load Model


In [274]:
import transformers
import torch
import json
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
model_llama = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

# **4. Evaluate Homework prompts**

## 4.1. Test response of Promtps

In [287]:
import json
import torch

# Load your JSON file and process each pair
with open('It-Subtask2-test.jsonl', 'r') as file:
    data = json.load(file)[:5]  # Load only the first 5 pairs
    for pair in data:
        id = pair['id']
        text = pair['text']
        choices = pair['choices']
        label_index = int(pair['label'])  # Convert label_index to an integer
        label = choices[label_index]  # Get the actual label using label_index

        # Format the prompt with text and hypothesis
        prompt_formatted = f"Qual è il tuo giudizio sull'accettabilità di {text}, su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente')."

        # Assuming you have tokenizer_llama and model_llama defined elsewhere
        input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
        output = model_llama(**input)
        logits = output.logits[0].detach().cpu().numpy()

        # Obtain the predicted label directly from logits
        probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
        print(f"Probabilities: {probabilities}")
        prediction_index = 1 if probabilities[1] > probabilities[0] else 0
        prediction_label = choices[prediction_index]
        prediction_confidence = probabilities[prediction_index] * 100
        prediction = {"label": prediction_label, "confidence": round(prediction_confidence, 1)}

        # Compare prediction with actual label
        is_correct = prediction_label == label

        # Visualize the formatted prompt and the prediction details
        print(f"ID: {id}")
        print("Prompt:", prompt_formatted)
        print("Prediction:", prediction)
        print("Actual Label:", label)
        print("Correct Prediction:", is_correct)
        print()


Probabilities: [0.01952113 0.9744667  0.00601211]
ID: 0
Prompt: Qual è il tuo giudizio sull'accettabilità di Amo gli arbusti, e le querce in generale., su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente').
Prediction: {'label': 'Pessimo', 'confidence': 97.4}
Actual Label: Sufficiente
Correct Prediction: False

Probabilities: [0.01955767 0.9729056  0.00753672]
ID: 1
Prompt: Qual è il tuo giudizio sull'accettabilità di Amo le montagne, e in particolare i centri commerciali., su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente').
Prediction: {'label': 'Ottimo', 'confidence': 97.3}
Actual Label: Pessimo
Correct Prediction: False

Probabilities: [0.02535288 0.9697725  0.00487464]
ID: 2
Prompt: Qual è il tuo giudizio sull'accettabilità di Amo le foreste più delle case., su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente').
Prediction: {'label': 'Ottimo', 'confidence': 97.0}
Actual Label: Ottimo
Correct Prediction: True

Probabilities: [

##4.2. Compare the diferent prompts

In [285]:
import json

total_correct = 0
total_pairs = 0

# Load the JSONL file and process each pair
with open('It-Subtask2-test.jsonl', 'r') as file:
    data = json.load(file) # Load only the first 5 pairs
    for pair in data:
        id = pair['id']
        text = pair['text']
        choices = pair['choices']
        label = int(pair['label']) 
        
        # Format the prompt with actual text and hypothesis
        prompt_formatted = f"Qual è il tuo giudizio sull'accettabilità di {text}, su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente')."

        input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
        output = model_llama(**input)
        logits = output.logits[0].detach().cpu().numpy()

        # Obtain the predicted label directly from logits
        probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
        prediction_index = 1 if probabilities[1] > probabilities[0] else 0
        prediction_label = choices[prediction_index]

        # Compare prediction with actual label
        is_correct = prediction_label == choices[label]

        # Accumulate statistics
        total_correct += is_correct
        total_pairs += 1

# Calculate overall statistics only if at least one pair was processed
if total_pairs > 0:
    accuracy = total_correct / total_pairs * 100

    # Print statistics
    print("Overall Statistics:")
    print("Total Pairs:", total_pairs)
    print("Total Correct:", total_correct)
    print("Accuracy:", round(accuracy, 2), "%")
else:
    print("No pairs found in the JSONL file.")


Overall Statistics:
Total Pairs: 1009
Total Correct: 257
Accuracy: 25.47 %


In [283]:
import json
import torch

def prompt_testing(prompt):
    total_correct = 0
    total_pairs = 0
    total_confidence = 0

    # Load the JSONL file and process each pair
    with open('It-Subtask2-train.jsonl', 'r') as file:
        data = json.load(file)  # Load only the first 5 pairs
        for pair in data:
            id = pair['id']
            text = pair['text']
            choices = pair['choices']
            label = int(pair['label'])  # Convert label_index to an integer
            
            # Format with text and hypothesis 
            prompt_formatted = prompt.replace('{{text}}', text)
            input = tokenizer_llama(prompt_formatted, return_tensors="pt").to(device)
            output = model_llama(**input)
            logits = output.logits[0].detach().cpu().numpy()

            # Obtain the predicted label directly from logits
            probabilities = torch.softmax(torch.tensor(logits), -1).detach().cpu().numpy()
            prediction_index = 1 if probabilities[1] > probabilities[0] else 0
            prediction_label = choices[prediction_index]

            # Compare prediction with actual label
            is_correct = prediction_label == choices[label]

            # Accumulate statistics
            total_correct += is_correct
            total_pairs += 1
            total_confidence += probabilities[prediction_index]

    # Calculate statistics 
    if total_pairs > 0:
        accuracy = total_correct / total_pairs * 100
        mean_confidence = total_confidence / total_pairs

        # Print statistics
        print("Total Pairs:", total_pairs)
        print("Total Correct:", total_correct)
        print("Accuracy:", round(accuracy, 2), "%")
        print("Mean Confidence:", round(mean_confidence, 2))
    else:
        print("No pairs found in the JSONL file.")

# Load prompts from the JSONL file and test each prompt
with open('prompts.jsonl', 'r') as prompts_file:
    prompt_data = json.load(prompts_file)
    for prompt_item in prompt_data:
        prompt = prompt_item["prompt"]
        print("\nPrompt:", prompt)
        prompt_testing(prompt)



Prompt: Qual è il tuo giudizio sull'accettabilità di {{text}}, su una scala da 1 a 7? ('1' indica 'Pessimo' e '7' indica 'Eccellente').
Total Pairs: 524
Total Correct: 149
Accuracy: 28.44 %
Mean Confidence: 0.98

Prompt: Considerando {{text}}, quanto lo trovi accettabile? Esprimi la tua valutazione su una scala da 1 a 7, dove '1' corrisponde a 'Pessimo' e '7' a 'Eccellente'.
Total Pairs: 524
Total Correct: 149
Accuracy: 28.44 %
Mean Confidence: 0.98

Prompt: Su una scala da 1 a 7, con '1' rappresentante 'Pessimo' e '7' 'Eccellente', quanto accettabile trovi {{text}}?
Total Pairs: 524
Total Correct: 149
Accuracy: 28.44 %
Mean Confidence: 0.83


**Prompt:** Quanto ritieni accettabile {{text}}, su una scala da 1 a 7, dove 1 è "Pessimo" e 7 è "Eccellente"?

Total Pairs: 1009

Total Correct: 257

Accuracy: 25.47 %

Mean Confidence: 0.98

**Prompt:** Considerando {{text}}, quanto lo valuteresti in termini di accettabilità, che va da "Pessimo" a "Eccellente"?

Total Pairs: 1009

Total Correct: 256

Accuracy: 25.37 %

Mean Confidence: 0.83

**Prompt:** Su una scala da 1 a 7, con "Pessimo" come 1 e "Eccellente" come 7, quanto accettabile trovi {{text}}?

Total Pairs: 1009

Total Correct: 257

Accuracy: 25.47 %

Mean Confidence: 0.7